In [1]:
#!/usr/bin/env python
"""
This script computes the Dirichlet domain for the Weeks manifold and its immediate neighbors,
refactored into a WeeksDirichlet class. It also builds a Cayley graph (via BFS) of the face–pairing
transformations (using the center (0,0,0) in the Klein model) up to a fixed radius and
adds it as a toggle-able component to the Plotly figure.
"""

from snappy import Manifold
import numpy as np
from plotly import graph_objects as go
import pyvista as pv

# --- Helper functions for coordinate conversions ---

def klein_to_minkowski(p):
    """
    Convert points in the Klein model (points inside the unit ball)
    to Minkowski 4-vectors on the hyperboloid.
    
    p: numpy array of shape (N,3) (or a single point shape (3,))
    Returns an array of shape (N,4) with first coordinate T.
    """
    p = np.atleast_2d(p)
    normsqr = (p**2).sum(axis=-1)
    # T = 1/sqrt(1 - |p|^2)
    w = 1 / np.sqrt(1 - normsqr)
    w = w.reshape(-1, 1)
    return np.concatenate([w, p * w], axis=-1)

def minkowski_to_klein(v):
    """
    Convert Minkowski 4-vectors to Klein coordinates.
    v: numpy array of shape (N,4); v[:, 0] is T.
    Returns an array of shape (N,3): (X/T, Y/T, Z/T).
    """
    return v[:, 1:] / v[:, 0].reshape(-1, 1)

def apply_minkowski_transform(T, V):
    """
    Apply a Minkowski isometry T (4×4 matrix) to vertices V given in Klein coordinates.
    
    Steps:
      1. Convert V (N,3) to Minkowski 4-vectors.
      2. Apply the transformation T.
      3. Convert back to Klein coordinates.
    
    V: numpy array of shape (N,3) or a single point shape (3,)
    """
    V = np.atleast_2d(V)
    V_mink = klein_to_minkowski(V)
    V_mink_trans = np.einsum('ij, kj -> ki', T, V_mink)
    return minkowski_to_klein(V_mink_trans)

# --- Helper function for mesh conversion ---
def pv_to_npmesh(pv_mesh):
    """
    Convert a PyVista mesh (from an STL file) to a dictionary with:
      'V': vertices (Nx3 numpy array)
      'F': faces (Mx3 numpy array of vertex indices; STL export is already triangulated)
    """
    vertices = pv_mesh.points
    faces = pv_mesh.faces.reshape(-1, 4)[:, 1:4]
    return {'V': vertices, 'F': faces}

# --- WeeksDirichlet Class ---
class WeeksDirichlet:
    def __init__(self, D):
        """
        Initialize from a SnapPy DirichletDomain D.
        This sets:
          - self.vertices from D.vertex_list()
          - self.edges from D.edge_list() (using tail and tip vertex indices)
          - self.mesh from an STL export (in the Klein model).
        """
        self.vertices = np.array(D.vertex_list()).astype(np.float64)
        self.edges = list(map(
            lambda x: (x['tail_vertex_index'], x['tip_vertex_index']),
            D.edge_list()))
        # Export the STL file (in the Klein model) and load it with PyVista.
        stl_filename = 'weeks_dirichlet.stl'
        D.export_stl(stl_filename, model='klein', cutout=False)
        pv_mesh = pv.read(stl_filename)
        self.mesh = pv_to_npmesh(pv_mesh)
    
    def transform_by(self, T):
        """
        Return a new WeeksDirichlet instance with all vertex data transformed by T.
        T: a 4×4 Minkowski transformation (ensure T is converted via np.array(list(T)).astype(np.float64)).
        The connectivity (edges and faces) remains the same.
        """
        new_obj = WeeksDirichlet.__new__(WeeksDirichlet)  # bypass __init__
        new_obj.vertices = apply_minkowski_transform(T, self.vertices)
        new_obj.edges = self.edges[:]  # same connectivity
        new_obj.mesh = dict(self.mesh)
        new_obj.mesh['V'] = apply_minkowski_transform(T, self.mesh['V'])
        new_obj.mesh['F'] = self.mesh['F']
        return new_obj
    
    def plot(self, group, color):
        """
        Return Plotly traces for this tile:
          - A Mesh3d trace for the bulk (with low opacity).
          - A Scatter3d trace for the vertices.
          - A Scatter3d trace for the edges (using the edge list from D.edge_list()).
        All traces are assigned the legend group `group`.
        """
        # Mesh trace: the bulk with lower opacity.
        mesh_trace = go.Mesh3d(
            x=self.mesh['V'][:, 0],
            y=self.mesh['V'][:, 1],
            z=self.mesh['V'][:, 2],
            i=self.mesh['F'][:, 0],
            j=self.mesh['F'][:, 1],
            k=self.mesh['F'][:, 2],
            color=color,
            opacity=0.3,  # more transparent bulk
            name=group,
            legendgroup=group,
            showlegend=True
        )
        # Vertex scatter: fully opaque markers.
        vertex_trace = go.Scatter3d(
            x=self.vertices[:, 0],
            y=self.vertices[:, 1],
            z=self.vertices[:, 2],
            mode='markers',
            marker=dict(size=4, color=color),
            name=f"{group} vertices",
            legendgroup=group,
            showlegend=False
        )
        # Edge scatter: use the reduced edge list.
        edge_x, edge_y, edge_z = [], [], []
        for (i, j) in self.edges:
            edge_x.extend([self.vertices[i, 0], self.vertices[j, 0], None])
            edge_y.extend([self.vertices[i, 1], self.vertices[j, 1], None])
            edge_z.extend([self.vertices[i, 2], self.vertices[j, 2], None])
        edge_trace = go.Scatter3d(
            x=edge_x,
            y=edge_y,
            z=edge_z,
            mode='lines',
            line=dict(color=color, width=2),
            name=f"{group} edges",
            legendgroup=group,
            showlegend=False
        )
        return [mesh_trace, vertex_trace, edge_trace]

# --- Cayley Graph Functions ---

def find_vertex(V, p, tol=1e-6):
    """
    Given a list V of vertices (each a numpy array of shape (3,)),
    return the index of a vertex that is within tol of p, or None if none found.
    """
    for idx, v in enumerate(V):
        if np.allclose(v, p, atol=tol):
            return idx
    return None

def cayley_graph(pairing_transformations, radius):
    """
    Perform a breadth-first search (BFS) on the Cayley graph defined by the pairing_transformations.
    The center is (0,0,0) in the Klein model. For each vertex, its neighbors are obtained by applying
    each pairing transformation.
    
    pairing_transformations: a list of face-pairing transformations (each a 4×4 matrix-like object)
    radius: the BFS radius (an integer)
    
    Returns a tuple G = (V, E):
      V: list of vertices (each a numpy array (3,))
      E: list of edges, each as a tuple (i, j) with i,j indices into V.
    """
    V = []  # list of vertices (numpy arrays of shape (3,))
    E = []  # list of edges (i, j)
    center = np.array([0.0, 0.0, 0.0])
    V.append(center)
    queue = [(0, 0)]  # each item is (vertex_index, depth)
    
    while queue:
        current_index, depth = queue.pop(0)
        if depth < radius:
            current_point = V[current_index]
            for T in pairing_transformations:
                # Convert T properly
                T_mat = np.array(list(T)).astype(np.float64)
                new_point = apply_minkowski_transform(T_mat, current_point.reshape(1,3))[0]
                j = find_vertex(V, new_point)
                if j is None:
                    j = len(V)
                    V.append(new_point)
                    queue.append((j, depth+1))
                # Add an undirected edge if not already added.
                edge = tuple(sorted((current_index, j)))
                if edge not in E:
                    E.append(edge)
    return V, E

def plot_graph(V, E, group="graph", color="black"):
    """
    Given a list of vertices V (Nx3) and edges E (list of (i,j) index pairs),
    return Plotly traces for the graph: a vertex scatter and an edge (line) scatter.
    The traces are assigned the legend group 'group'.
    """
    V = np.array(V)
    vertex_trace = go.Scatter3d(
        x=V[:,0],
        y=V[:,1],
        z=V[:,2],
        mode='markers',
        marker=dict(size=4, color=color),
        name=f"{group} vertices",
        legendgroup=group,
        showlegend=True
    )
    edge_x, edge_y, edge_z = [], [], []
    for i, j in E:
        edge_x.extend([V[i,0], V[j,0], None])
        edge_y.extend([V[i,1], V[j,1], None])
        edge_z.extend([V[i,2], V[j,2], None])
    edge_trace = go.Scatter3d(
        x=edge_x,
        y=edge_y,
        z=edge_z,
        mode='lines',
        line=dict(color=color, width=2),
        name=f"{group} edges",
        legendgroup=group,
        showlegend=False
    )
    return [vertex_trace, edge_trace]

# --- Main Computation and Visualization ---

# 1. Compute the Dirichlet domain for the Weeks manifold.
M = Manifold("m003(-3,1)")
D = M.dirichlet_domain(include_words=True)
print("Dirichlet Domain Info:")
print(D)

# 2. Create the central tile as an instance of WeeksDirichlet.
center_tile = WeeksDirichlet(D)

# 3. Get the face-pairing (neighboring) transformations.
pairing_mats = list(D.pairing_matrices())
print("Number of face-pairing transformations (neighbors):", len(pairing_mats))

# 4. Create neighbor tiles by applying each pairing transformation.
neighbor_tiles = []
for idx, T in enumerate(pairing_mats):
    # Use np.array(list(T)).astype(np.float64) for correct conversion.
    T_mat = np.array(list(T)).astype(np.float64)
    neighbor_tile = center_tile.transform_by(T_mat)
    neighbor_tiles.append(neighbor_tile)

# 5. Create Plotly traces for the Dirichlet domain tiles.
all_traces = []
# Central tile: group "center" and color red.
all_traces.extend(center_tile.plot("center", "red"))

# 6. Create the Cayley graph.
radius = 2  # you can adjust this radius as desired
V_graph, E_graph = cayley_graph(pairing_mats, radius)
graph_traces = plot_graph(V_graph, E_graph, group="graph", color="black")
all_traces.extend(graph_traces)

# Neighbors: group names "1", "2", … with different colors.
colors = ['blue', 'green', 'purple', 'orange', 'cyan', 'magenta', 
          'yellow', 'pink', 'gray', 'brown', 'olive', 'teal', 'lavender', 'gold']
for idx, tile in enumerate(neighbor_tiles):
    group = str(idx+1)
    color = colors[idx % len(colors)]
    all_traces.extend(tile.plot(group, color))

# 7. Create the Plotly figure and display it.
fig = go.Figure(data=all_traces)
fig.update_layout(
    scene=dict(aspectmode='data'),
    title="Weeks Manifold (Klein Model): Dirichlet Domain, neighbors, and Cayley Graph (r=2)")
fig.write_html("weeks_tiling.html")


Dirichlet Domain Info:
26 finite vertices, 0 ideal vertices; 42 edges; 18 faces
Number of face-pairing transformations (neighbors): 18
